# Shut the Box

## Step 0: Utils for Jupyter Notebooks

### Add button to Show/Hide code (to focus on outputs)

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show Code"></form>''')

### Add magic command to store python variables in csv file

In [2]:
# credit to sirex, https://github.com/ipython/ipython/issues/6701
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Step 1: Generate models

In [35]:
# Edit this cell to define parameters for your model...

boards = 12 # Number of boards to use
sides = 12 # Number of sides per die
die = 1 # Number of die to roll each round

max_score = boards * (boards+1) // 2

In [36]:
%%writetemplate ShutTheBox/params/stb_params_high.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 1

In [37]:
%%writetemplate ShutTheBox/params/stb_params_low.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 2

In [38]:
%%writetemplate ShutTheBox/params/stb_nondet_params.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 0

In [39]:
!mkdir test

A subdirectory or file test already exists.


In [40]:
model_spec = f"stb{boards}_{die}d{sides}"

!mkdir "ShutTheBox/models/$model_spec"
output_model_high = f"ShutTheBox/models/{model_spec}/{model_spec}_high.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_high ShutTheBox/params/stb_params_high.csv

A subdirectory or file ShutTheBox/models/stb12_1d12 already exists.


Model generated successfully at ShutTheBox\models\stb12_1d12\stb12_1d12_high.prism


In [41]:
output_model_low = f"ShutTheBox/models/{model_spec}/{model_spec}_low.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_low ShutTheBox/params/stb_params_low.csv

Model generated successfully at ShutTheBox\models\stb12_1d12\stb12_1d12_low.prism


In [42]:
output_model_nondet = f"ShutTheBox/models/{model_spec}/{model_spec}_nondet.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_nondet ShutTheBox/params/stb_nondet_params.csv

Model generated successfully at ShutTheBox\models\stb12_1d12\stb12_1d12_nondet.prism


In [43]:
output_props = f"ShutTheBox/props/{model_spec}.props"

!python utils/pyprism.py ShutTheBox/stbprops.pyprism $output_props ShutTheBox/params/stb_params_high.csv

Model generated successfully at ShutTheBox\props\stb12_1d12.props


## Step 2: Check properties, run experiments, export data

### Does the game eventually always terminate?

We expect to see a value of approximately 1 here (it won't be exact because CSGs only support methods based on numerical convergence, so it may be slightly under or even over 1!

In [44]:
!prism $output_model_high $output_props -prop 1 | grep -E "(Model checking)|(Result)|(sec)|(Error)"

Model checking: Pmax=? [ F game_over ]
Reachability (BFS): 20 iterations in 0.03 seconds (average 0.001400, setup 0.00)
Time for model construction: 0.18 seconds.
Prob0A: 7 iterations in 0.01 seconds (average 0.002143, setup 0.00)
Prob1E: 8 iterations in 0.01 seconds (average 0.000625, setup 0.00)
Time for model checking: 0.021 seconds.
Result: 1.0 (value in the initial state)


### Probability of obtaining each score 

In [13]:
data_location = f"ShutTheBox/data/{model_spec}"

!mkdir "$data_location"

prob_score_high_file = f"{data_location}/{model_spec}_prob_score_high.csv"

!prism $output_model_high $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_high_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

A subdirectory or file ShutTheBox/data/stb12_2d6 already exists.


Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 3.858024691358024E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 9.430727023319616E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=8
Result: 0.0011788408779149521 (value in the initial state)
Model checking: P

In [14]:
prob_score_low_file = f"{data_location}/{model_spec}_prob_score_low.csv"

!prism $output_model_low $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_low_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(sec)|(Error)"

Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=1
Reachability (BFS): 22 iterations in 0.02 seconds (average 0.000773, setup 0.00)
Time for model construction: 0.189 seconds.
Time for model checking: 0.016 seconds.
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=2
Prob0E: 6 iterations in 0.01 seconds (average 0.001000, setup 0.00)
Prob1A: 3 iterations in 0.01 seconds (average 0.001667, setup 0.00)
Iterative method: 4 iterations in 0.00 seconds (average 0.000000, setup 0.00)
Time for model checking: 0.016 seconds.
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=3
Prob0E: 6 iterations in 0.00 seconds (average 0.000500, setup 0.00)
Prob1A: 3 iterations in 0.00 seconds (average 0.000667, setup 0.00)
Iterative method: 4 iterations in 0.00 seconds (average 0.000000, setup 0.00)
Time for model checking: 0.01 seconds.
Result: 0.001543

In [15]:
prob_score_nondet_min_file = f"{data_location}/{model_spec}_prob_score_nondet_min.csv"

!prism $output_model_nondet $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_nondet_min_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 1.2860082304526747E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 3.858024691358024E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 4.715363511659808E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=8
Result: 0.0010716735253772291 (value in the initial state)
Model checking: P

In [16]:
prob_score_nondet_max_file = f"{data_location}/{model_spec}_prob_score_nondet_max.csv"

!prism $output_model_nondet $output_props -prop 3 -const k=1:{max_score} -exportresults {prob_score_nondet_max_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.0015432098765432098 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.004629629629629629 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.009516460905349794 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.012217078189300411 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.020233196159122085 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=8
Result: 0.023040980795610427 (value in the

### Probability distribution of scores with max expected value strategy

In [45]:
!prism $output_model_nondet $output_props -prop 8  -s -exportadvmdp ShutTheBox/data/{model_spec}/{model_spec}.tra -exportstates ShutTheBox/data/{model_spec}/{model_spec}.sta -exportlabels ShutTheBox/data/{model_spec}/{model_spec}.lab -exportstaterewards ShutTheBox/data/{model_spec}/{model_spec}.srew -exporttransrewards ShutTheBox/data/{model_spec}/{model_spec}.trew

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Fri Mar 26 13:01:04 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games ShutTheBox/models/stb12_1d12/stb12_1d12_nondet.prism ShutTheBox/props/stb12_1d12.props -prop 8 -s -exportadvmdp ShutTheBox/data/stb12_1d12/stb12_1d12.tra -exportstates ShutTheBox/data/stb12_1d12/stb12_1d12.sta -exportlabels ShutTheBox/data/stb12_1d12/stb12_1d12.lab -exportstaterewards ShutTheBox/data/stb12_1d12/stb12_1d12.srew -exporttransrewards ShutTheBox/data/stb12_1d12/stb12_1d12.trew

Parsing model file "ShutTheBox/models/stb12_1d12/stb12_1d12_nondet.prism"...

Type:        MDP
Modules:     player1 board1 board2 board3 board4 board5 board6 board7 board8 board9 board10 board11 board12 
Variables:   state die b1 b2 b3 b4 b5 b6 b7 b8 b9 b10 b11 b12 

Parsing properties file "ShutTheBox/props/stb12_1d12.props"...

22 properties:
(1) Pmax=? [ F game_over ]
(2) Pmin=? [ F game_over&score=k ]
(3) Pmax=? [ F game_over

In [46]:
avg_max_file = f"ShutTheBox/data/{model_spec}/{model_spec}_prob_score_avg_max.csv"

!prism -importmodel ShutTheBox/data/{model_spec}/{model_spec}.all -mdp -pf "const int k; Pmax=? [ F state=3&(b1*1 + b2*2 + b3*3 + b4*4 + b5*5 + b6*6 + b7*7 + b8*8 + b9*9 + b10*10 + b11*11 + b12*12)=k ]" -const k=1:{max_score} -exportresults {avg_max_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=1
Result: 0.00694443888889 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=2
Result: 0.00694443888889 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=3
Result: 0.002314812037038148 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=4
Result: 0.002314812037038148 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=5
Result: 0.004629624074076296 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6

### Probability distribution of scores with min expected value strategy

In [47]:
!prism $output_model_nondet $output_props -prop 7  -s -exportadvmdp ShutTheBox/data/{model_spec}/{model_spec}_min.tra -exportstates ShutTheBox/data/{model_spec}/{model_spec}_min.sta -exportlabels ShutTheBox/data/{model_spec}/{model_spec}_min.lab -exportstaterewards ShutTheBox/data/{model_spec}/{model_spec}_min.srew -exporttransrewards ShutTheBox/data/{model_spec}/{model_spec}_min.trew

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Fri Mar 26 13:01:33 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games ShutTheBox/models/stb12_1d12/stb12_1d12_nondet.prism ShutTheBox/props/stb12_1d12.props -prop 7 -s -exportadvmdp ShutTheBox/data/stb12_1d12/stb12_1d12_min.tra -exportstates ShutTheBox/data/stb12_1d12/stb12_1d12_min.sta -exportlabels ShutTheBox/data/stb12_1d12/stb12_1d12_min.lab -exportstaterewards ShutTheBox/data/stb12_1d12/stb12_1d12_min.srew -exporttransrewards ShutTheBox/data/stb12_1d12/stb12_1d12_min.trew

Parsing model file "ShutTheBox/models/stb12_1d12/stb12_1d12_nondet.prism"...

Type:        MDP
Modules:     player1 board1 board2 board3 board4 board5 board6 board7 board8 board9 board10 board11 board12 
Variables:   state die b1 b2 b3 b4 b5 b6 b7 b8 b9 b10 b11 b12 

Parsing properties file "ShutTheBox/props/stb12_1d12.props"...

22 properties:
(1) Pmax=? [ F game_over ]
(2) Pmin=? [ F game_over&score=k ]
(3) 

In [48]:
avg_min_file = f"ShutTheBox/data/{model_spec}/{model_spec}_prob_score_avg_min.csv"

!prism -importmodel ShutTheBox/data/{model_spec}/{model_spec}_min.all -mdp -pf "const int k; Pmax=? [ F state=3&(b1*1 + b2*2 + b3*3 + b4*4 + b5*5 + b6*6 + b7*7 + b8*8 + b9*9 + b10*10 + b11*11 + b12*12)=k ]" -const k=1:{max_score} -exportresults {avg_min_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=1
Result: 0.00694443888889 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=2
Result: 0.00694443888889 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=3
Result: 0.016203689814818147 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=4
Result: 0.015046283796299072 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6*6+b7*7+b8*8+b9*9+b10*10+b11*11+b12*12)=k ]
Property constants: k=5
Result: 0.016203689814818147 (value in the initial state)
Model checking: Pmax=? [ F state=3&(b1*1+b2*2+b3*3+b4*4+b5*5+b6

### Expected number of die rolls

In [21]:
print("High board strategy:")
!prism $output_model_high $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"
print("Low board strategy:")
!prism $output_model_low $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"

High board strategy:
Model checking: R{"no_rolls"}max=? [ F game_over ]
Result: 7.006172038789394 (value in the initial state)
Low board strategy:
Model checking: R{"no_rolls"}max=? [ F game_over ]
Result: 4.219568107857559 (value in the initial state)


### Probability of obtaining a score given a particular board is covered

In [22]:
for covered_board in range(1, boards+1):
    !prism $output_model_high $output_props -prop {10+covered_board} -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_score_with_covered_{covered_board}.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=6
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=8
Result: 9.645061728395061E-4 (value in the initial state)
Model checking: Pmin=? [ F ga

Model checking: Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=5
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=6
Result: 3.858024691358024E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=7
Result: 1.7146776406035664E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=8
Result: 2.1433470507544578E-4 (valu

Model checking: Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=5
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=6
Result: 2.5720164609053495E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=7
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=8
Result: 9.645061728395061E-4 (value in the initial s

Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=6
Result: 1.2860082304526747E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=7
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=8
Result: 9.645061728395061E-4 (value in the initial state)
Model checki

Model checking: Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=7
Result: 1.7146776406035664E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b5=1&scor

...

Model checking: Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b8=1&score=k ]
Property con

Model checking: Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b9=1&score=k ]
Property con

Model checking: Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b10=1&score=k ]
Pro

Model checking: Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b11=1&score=k ]
Pro

Model checking: Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b12=1&score=k ]
Pro

### Cumulative probability of a score

In [23]:
!prism $output_model_high $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_high.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.0010288065843621398 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.0014146090534979422 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.002357681755829904 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=8
Result: 0.003536522633744856 (val

In [24]:
!prism $output_model_low $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_low.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.0023148148148148147 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.006944444444444444 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.01633230452674897 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.02854938271604938 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.047925240054869686 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=8
Result: 0.06957304526748971 (value i

In [25]:
!prism $output_model_nondet $output_props -prop 5 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_min.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=3
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=4
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.0010288065843621398 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.0014146090534979422 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.002357681755829904 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=8
Result: 0.003536522633744856 (val

In [26]:
cum_prob_max_file = "ShutTheBox/data/cum_prob_max.csv"

!prism $output_model_nondet $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_max.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 7.716049382716049E-4 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.0023148148148148147 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.006944444444444444 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.01633230452674897 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.02854938271604938 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.04826817558299039 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=8
Result: 0.07077331961591221 (value in

# Optimal Strategy for highest expected score

In [27]:
!prism $output_model_nondet $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: R{"total_boards"}max=? [ F game_over ]
Result: 42.91857934707854 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_min.csv"...


In [28]:
!prism -mdp -importtrans ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.tra -importstates ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.sta -importlabels ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.lab -exporttrans test.tra -exportstates test.sta -exporttransdotstates test.dot -fixdl

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Fri Mar 26 12:03:17 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games -mdp -importtrans ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_min.tra -importstates ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_min.sta -importlabels ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_min.lab -exporttrans test.tra -exportstates test.sta -exporttransdotstates test.dot -fixdl

Importing model from "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_min.tra", "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_min.sta", "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_min.lab"...
Using specified model type: MDP

Building model...

Computing reachable states...

Reachability (BFS): 24 iterations in 0.02 seconds (average 0.000833, setup 0.00)

Time for model construction: 13.915 seconds.


Type:        MDP
States:      10525 (1 initial)
Transitions: 26135
Choices:     10525

Transition matrix: 10603 nodes (8 termina

## Try for highboard

In [29]:
!prism $output_model_high $output_props -prop 7  -s -exportadvmdp ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.tra -exportstates ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.sta -exportlabels ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.lab -exportstaterewards ShutTheBox/data/{model_spec}/state_rewards_high.txt -exporttransrewards ShutTheBox/data/{model_spec}/{model_spec}_trans_rewards_high.txt

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Fri Mar 26 12:03:32 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games ShutTheBox/models/stb12_2d6/stb12_2d6_high.prism ShutTheBox/props/stb12_2d6.props -prop 7 -s -exportadvmdp ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_highboard.tra -exportstates ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_highboard.sta -exportlabels ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_highboard.lab -exportstaterewards ShutTheBox/data/stb12_2d6/state_rewards_high.txt -exporttransrewards ShutTheBox/data/stb12_2d6/stb12_2d6_trans_rewards_high.txt

Parsing model file "ShutTheBox/models/stb12_2d6/stb12_2d6_high.prism"...

Type:        MDP
Modules:     player1 board1 board2 board3 board4 board5 board6 board7 board8 board9 board10 board11 board12 
Variables:   state die b1 b2 b3 b4 b5 b6 b7 b8 b9 b10 b11 b12 

Parsing properties file "ShutTheBox/props/stb12_2d6.props"...

22 properties:
(1) Pmax=? [ F game_

In [30]:
!prism -mdp -importtrans ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.tra -importstates ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.sta -importlabels ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.lab -exporttrans test_highboard.tra -exportstates test_highboard.sta -exporttransdotstates test_highboard.dot -fixdl

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Fri Mar 26 12:03:33 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games -mdp -importtrans ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_highboard.tra -importstates ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_highboard.sta -importlabels ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_highboard.lab -exporttrans test_highboard.tra -exportstates test_highboard.sta -exporttransdotstates test_highboard.dot -fixdl

Importing model from "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_highboard.tra", "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_highboard.sta", "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_highboard.lab"...
Using specified model type: MDP

Building model...

Computing reachable states...

Reachability (BFS): 22 iterations in 0.04 seconds (average 0.001727, setup 0.00)

Time for model construction: 9.688 seconds.


Type:        MDP
States:      82745 (1 initial)
Transitions: 28

In [31]:
!prism $output_model_nondet $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_max.csv:csv -s | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: R{"total_boards"}max=? [ F game_over ]
Result: 42.91857934707854 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_max.csv"...


In [32]:
!prism $output_model_low $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_low.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: R{"total_boards"}max=? [ F game_over ]
Result: 24.30185396277705 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_low.csv"...


In [33]:
!prism $output_model_high $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_high.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: R{"total_boards"}max=? [ F game_over ]
Result: 42.8087697181286 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_high.csv"...
